# Controlled Scenario

This notebook contains the experiment to **reproduce the observed course of COVID19 under the taken interventions** during the lockdown in Germany and Switzerland.

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
import numpy as np
import pandas as pd
import pickle
import multiprocessing

In [ ]:
from lib.measures import *
from lib.experiment import run_experiment, save_summary, load_summary
from lib.calibration_settings import settings_lockdown_dates, calibration_setting_paths

## 1. General settings  

#### Set the random seed for reproducibility

In [ ]:
# Choose random seed
c = 0
np.random.seed(c)
TO_HOURS = 24.0

# Define prefix string used to save plots
expstr = 'controlled-scenario'

#### Set the number of roll-outs to simulate

In [ ]:
random_repeats = 2 # Set to at least 40 to obtain stable results

#### Define locations, mobility settings used for simulation, and experiment parameters

In [ ]:
locs = {
    'GER': {
        'TU': 'lib/mobility/Tubingen_settings_10.pk',
        'KL': 'lib/mobility/Kaiserslautern_settings_10.pk',
        'RH': 'lib/mobility/Ruedesheim_settings_10.pk', 
        'TR': 'lib/mobility/Tirschenreuth_settings_10.pk', 
    },
    'CH': {
        'VD': 'lib/mobility/Lausanne_settings_10.pk', 
        'LU': 'lib/mobility/Lucerne_settings_5.pk', 
        'TI': 'lib/mobility/Locarno_settings_2.pk', 
        'JU': 'lib/mobility/Jura_settings_10.pk',
    }
}

## 2. Run the simulations

Use settings as above and simulate in the future with additional measures

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [ ]:
for country, areas in locs.items():
    for area, mob_settings in areas.items():
        s = []
        
        # start simulation when calibration started
        start_date = calibration_setting_paths[country][area][1] # start date of calibration
        lockdown_date = settings_lockdown_dates[country]['start'] # start date of lockdown
        end_date = settings_lockdown_dates[country]['end'] # end date of lockdown        
        
        # calibrated measures 
        days_until_lockdown = (pd.to_datetime(lockdown_date) - pd.to_datetime(start_date)).days
        max_time = (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days
        
        p_stay_home = 0.0 # TBD
        beta_multiplier = { 
            'education': 0.0, 
            'social': 0.0, 
            'bus_stop': 1.0, 
            'office': 0.0, 
            'supermarket': 1.0
        }
        
        m = [
            BetaMultiplierMeasureByType(
                t_window=Interval(days_until_lockdown * TO_HOURS, max_time * TO_HOURS), 
                beta_multiplier=beta_multiplier),
            SocialDistancingForAllMeasure(
                t_window=Interval(days_until_lockdown * TO_HOURS, max_time * TO_HOURS), 
                p_stay_home=p_stay_home)
        ]
        
        # simulation
        baseline = run_experiment(
            country=country, 
            area=area, 
            mob_settings=mob_settings,
            start_date=start_date, 
            end_date=end_date, 
            measure_list=m, 
            random_repeats=random_repeats,
            test_update=None, 
            seed_summary_path=None)
        s.append(baseline)
        
        print(country, area, ' done.', flush=True)
        
        # save results
        save_summary(s, f'state-{expstr}--{country}-{area}.pk')

## 3. Plot the results

Import libs

In [ ]:
from lib.plot import Plotter

Create figures from summaries

In [ ]:
for country, areas in locs.items():
    for area in areas:
        s = load_summary(f'state-{expstr}--{country}-{area}.pk')
        plotter = Plotter()
        titles = ['Observed measures'],
        fn = f'{expstr}--{country}-{area}'
        print(fn)
        plotter.compare_total_infections(
            s, 
            titles=titles, 
            filename=fn, 
            start_date=calibration_setting_paths[country][area][1], # start date of calibration
            figsize=(6,4), acc=500, 
            ymax=500, errorevery=14)